In [23]:
import cv2
import random
import numpy as np
import math
import os
import statistics
from PIL import Image, ImageOps 
import matplotlib.pyplot as plt
from collections import Counter
from scipy.spatial import distance
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

In [24]:
def hog_vector():
    dict_hog = {}
    for d in range(0, 180, +20):
        dict_hog[d] = 0
    for e in range(temp_np_grad.shape[0]):
        for f in range(temp_np_grad.shape[1]):
            curr_grad = temp_np_grad[e][f]
            curr_magn = temp_np_magn[e][f]
            list_diff = []
            bin_nearest = 160
            for g in list_bins:
                val_dist_bin = curr_grad - g
                if val_dist_bin < 20:
                    bin_nearest = g
                    break

            if val_dist_bin == 0:
                dict_hog[bin_nearest] = dict_hog[bin_nearest] + curr_magn
            else:
                if bin_nearest == 160:
                    part_1 = ((180 - curr_grad) / 20) * curr_magn
                    part_2 = ((curr_grad - 160) / 20) * curr_magn
                    dict_hog[160] += part_1
                    dict_hog[0] += part_2
                else:
                    bin_next_near = bin_nearest + 20
                    part_1 = ((bin_next_near - curr_grad) / 20) * curr_magn
                    part_2 = ((curr_grad - bin_nearest) / 20) * curr_magn
                    dict_hog[bin_nearest] += part_1
                    dict_hog[bin_next_near] += part_2
    
    return dict_hog

list_directory = ['Car_Q2\\', 'Dog_Q2\\']
list_tags = ['Car', 'Dog']
list_X = []
list_Y = []

for af in range(len(list_directory)):
    directory = list_directory[af]
    curr_tag = list_tags[af]
    docs = []
    for entry in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, entry)):
            docs.append(entry)
            doc_loc = os.path.join(directory, entry)

    for d in docs:
        cmp_path = os.path.join(directory, d)
        img_read = Image.open(cmp_path) 
        img_read = img_read.resize((256, 256))
        img_gray = ImageOps.grayscale(img_read)
        img_array = np.array(img_gray)
        temp_pad = np.pad(img_array, pad_width=1, mode='constant', constant_values=0)
        no_rows = temp_pad.shape[0]
        no_col = temp_pad.shape[1]

        list_magnitude = []
        list_gradient = []
        for i in range(1, no_rows-1):
            list_temp_1 = []
            list_temp_2 = []
            for j in range(1, no_col-1):
                y_diff = temp_pad[i-1][j] - temp_pad[i+1][j]
                x_diff = temp_pad[i][j+1] - temp_pad[i][j-1]
                val_magnitude = math.sqrt((x_diff ** 2) + (y_diff ** 2))
                val_gradient = math.degrees(math.atan(abs(y_diff) / abs(x_diff)))
                list_temp_1.append(val_magnitude)
                list_temp_2.append(val_gradient)

            list_magnitude.append(list_temp_1)
            list_gradient.append(list_temp_2)


        np_magn = np.array(list_magnitude)
        np_grad = np.array(list_gradient)
        
        patch_rows = 8
        patch_cols = 8

        end_row = np_grad.shape[0] - patch_rows
        end_col = np_grad.shape[1] - patch_cols
        list_bins = [r for r in range(0, 180, +20)]
        list_list_dict = []

        for a in range(0, end_row + 1, +8):
            slice_till_row = a + patch_rows - 1
            list_dict = []
            for b in range(0, end_col + 1, +8):
                slice_till_col = b + patch_cols - 1
                temp_np_grad = np_grad[a:slice_till_row+1, b:slice_till_col+1]
                temp_np_magn = np_magn[a:slice_till_row+1, b:slice_till_col+1]

                #Now finding the bucket range to which the gradient value belongs
                dict_hog_vector = hog_vector()
                list_dict.append(dict_hog_vector)
            list_list_dict.append(list_dict)

        list_vector_image = []
        count_row_list_list = len(list_list_dict)
        count_cols_list_list = len(list_list_dict[0])
        #Now making vectors of 16 X 16 patches after normalization
        for r_no in range(0, count_row_list_list - 1):
            for c_no in range(0, count_cols_list_list - 1):
                list_patch_vector = []
                for c_r in range(r_no, r_no + 2):
                    for c_c in range(c_no, c_no + 2):
                        list_hog_vec = list(list_list_dict[c_r][c_c].values())
                        list_patch_vector.extend(list_hog_vec)

                np_patch_vector = np.array(list_patch_vector)
                norm_denominator = np.linalg.norm(np_patch_vector)
                patch_vector_normalised = [hj / norm_denominator for hj in list_patch_vector]
                list_vector_image.extend(patch_vector_normalised)
                
        list_X.append(list_vector_image)
        list_Y.append(curr_tag)
        

In [25]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
np_list_X = np.array(list_X)
np_list_Y = np.array(list_Y)
X_train, X_test, y_train, y_test = train_test_split(np_list_X, np_list_Y, test_size=0.33)
X_train = np.nan_to_num(X_train)
# print(type(X_train))
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

# clf = svm.SVC()
# clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [26]:
X_test = np.nan_to_num(X_test) 
# y_pred = clf.predict(X_test)
y_pred = neigh.predict(X_test)
pred_accuracy = accuracy_score(y_test, y_pred)
print(f"The accuracy = {(pred_accuracy * 100)}%")

The accuracy = 62.5%
